<a href="https://colab.research.google.com/github/ChiaraSolito/ComputerVision/blob/main/exercises1_lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 02 - 1 #

## Image classification on the Fashion-MNIST dataset using a ResNet-18 ##

**1**

* Create a custom `FMnistResNet18` class in which:
    * Download the pre-trained ResNet-18
    * Change the first and last layers

Specifically, the input and output layers of a pre-trained ResNet-18 need to be changed, since ResNet was originally designed for ImageNet competition, which was a color (3-channel) image classification task with 1000 classes. Fashon-MNIST, on the other hand, only contains 10 classes, and it’s images are in the grayscale (*i.e.*,1-channel).

In [ ]:
import torch
import torch.optim
import torchvision

import torch.nn as nn
import torchvision.models as models

# from tqdm import tqdm
from tqdm.notebook import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import ResNet18_Weights


# Hyperparameters.
LR = 3e-4
EPOCH = 5
BATCH_SIZE = 50
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
class FMnistResNet18(nn.Module):
    def __init__(self, in_channels=1):
        super(FMnistResNet18, self).__init__()

        # Load the pre-trained ResNet-18 model from torchvision.models.
        # TODO.

        # Change the input layer to take grayscale images, instead of RGB images.
        # TODO.

        # Change the output layer to output 10 classes instead of 1000 classes.
        # TODO.

    def forward(self, x):
        return self.model(x)

# Test the network, and verify the layers.
test_my_resnet = FMnistResNet18()
# [N, C, H, W]: batch N, channels C, depth D, height H, width W.
dummy_input = torch.randn((32, 1, 244, 244))
output = test_my_resnet(dummy_input)
print(output.shape)

# print(test_my_resnet)

**2**

* Create DataLoaders
    * Hint: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
* Define the model
* Define the loss function
* Define the optimizer

In [ ]:
# Dataset.
fashion_mnist = torchvision.datasets.FashionMNIST(download=True,
                                                  train=True,
                                                  root=".").train_data.float()

# Transformations.
data_transform = transforms.Compose([transforms.Resize((224, 224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize((fashion_mnist.mean()/255,), (fashion_mnist.std()/255,))])

# DataLoaders.
# TODO.

# Define the model.
# TODO.

# Define the loss function.
# TODO.

# Define the optimizer.
# TODO.

**3**

* Write the training step
    * Hint: https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

In [ ]:
losses = []

# Training step.
print(f"Start training on {DEVICE} [...]")

for e in range(EPOCH):
    e_loss = 0.0

    for i, data in (tepoch := tqdm(enumerate(), unit="batch", total=len())):
        # TODO.
        pass

**4**

* Write the evaluation step

In [ ]:
# Evaluation step.
t_loss = 0
correct = 0

with torch.no_grad():
    for i, data in (tepoch := tqdm(enumerate(), unit="batch", total=len())):
        # TODO.
        pass